In [1]:
import bz2
import tarfile
import pandas as pd

df = None

jp_sentences_file = './jpn_sentences.tsv.bz2'
attribution_file = './sentences_base.tar.bz2'
attribution_file_decompressed = '.' + attribution_file.split('.')[-3] + '.csv'

# read jp sentences
with bz2.open(jp_sentences_file, 'rt') as file:
    df = pd.read_csv(file, delimiter='\t', header=None, names=['tatoeba_id','language','sentence'])
# read attribution (translation or original)
with tarfile.open(attribution_file, 'r:bz2') as tar:
    tar.extractall()
df_attribution = pd.read_csv(attribution_file_decompressed, delimiter='\t', header=None, names=['tatoeba_id', 'base_field'])

/tmp/ipykernel_68351/1506600146.py:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_attribution = pd.read_csv(attribution_file_decompressed, delimiter='\t', header=None, names=['tatoeba_id', 'base_field'])


In [2]:
df.language.value_counts()

language
jpn    239380
Name: count, dtype: int64

In [3]:
# we don't really care about the sentence link so we delete the numbers > 0 (we keep 0, non0, and unknown)

def process_base_field(field):
    if field == '0':
        return 'original'
    elif field == '\\N':
        return 'unknown'
    else: return 'translation'

df_attribution['base_field'] = df_attribution['base_field'].map(process_base_field)

In [4]:
# we only care about the jp sentences
df_attribution_jp = df_attribution[df_attribution['tatoeba_id'].isin(df['tatoeba_id'])]
df_attribution_jp.shape, df.shape

((239380, 2), (239380, 3))

In [5]:
#(df_attribution_jp['base_field'] == '0').value_counts()

In [5]:
# merge the information
df = df.merge(df_attribution_jp, on='tatoeba_id')
# add the source link
df['source'] = 'https://tatoeba.org/en/downloads'

In [6]:
info_columns = ['source', 'tatoeba_id', 'base_field']
df['info'] = df[info_columns].to_dict(orient='records')

In [7]:
# drop language and other info
df.drop('language', axis='columns', inplace=True)
df_clean = df.drop(info_columns, axis='columns')
df_sentence_only = df_clean['sentence']

df_clean.to_csv('tatoeba_info.csv')
df_sentence_only.to_csv('tatoeba.csv')

df_clean

,sentence,info
0,きみにちょっとしたものをもってきたよ。,"{'source': 'https://tatoeba.org/en/downloads',..."
1,何かしてみましょう。,"{'source': 'https://tatoeba.org/en/downloads',..."
2,私は眠らなければなりません。,"{'source': 'https://tatoeba.org/en/downloads',..."
3,何してるの？,"{'source': 'https://tatoeba.org/en/downloads',..."
4,今日は６月１８日で、ムーリエルの誕生日です！,"{'source': 'https://tatoeba.org/en/downloads',..."
...,...,...
239375,ワインをお願い。,"{'source': 'https://tatoeba.org/en/downloads',..."
239376,彼らは私の話を信じようとしなかった。,"{'source': 'https://tatoeba.org/en/downloads',..."
239377,ハロウィンの仮装だよ。,"{'source': 'https://tatoeba.org/en/downloads',..."
239378,トムの誕生日に、新しい自転車をプレゼントするんだ。,"{'source': 'https://tatoeba.org/en/downloads',..."


In [8]:
df['base_field'].value_counts()

base_field
unknown        148005
translation     81809
original         9566
Name: count, dtype: int64

In [9]:
#pd.json_normalize(df['info'])